# <i class="fa fa-folder-open" style="font-size: 2em;"></i> Arterial stiffness - Microstructure & Perfusion of the corpus callosum 
***


<div class="alert alert-info" role="alert">
<p style="font-size:16px;"><strong>This notebook contains the code written in R to perform the statistical analysis and plots of the following study:</strong></p>

<br>
<a href="" target="blank">Arterial stiffness and corpus callosum integrity</a>

</div>    

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/37/Plotly-logo-01-square.png/220px-Plotly-logo-01-square.png" width="130" height="130" align="right" />

<img src="https://usercontent2.hubstatic.com/13881141.png" width="130" height="130" align="right" />

# <i class="fa fa-compass" style="font-size: 1.5em;"></i><left> &nbsp; Load all necessary libraries. </left> 

### R packages are already installed on this BinderHub instance via [ `install.R`](https://github.com/atefbadji/cfPWV_Microstructure-Perfusion_CorpusCallosum-/blob/master/install.R)

<p style="color:blue;"><strong>Please run the cell below to load packages.</strong><p>

In [ ]:
# ppcor to perform partial correlations
library("MASS")
library("ppcor")
# ggplot and plotly for data visualization 
suppressMessages(library("ggplot2"))
suppressMessages(library("plotly"))
library("gridExtra")
# olssr to perform stepwise linear regression
suppressMessages(library("olsrr"))
print("Success: All the packages have been loaded.")


<div class="alert alert-warning" role="alert">
<p style="font-size:20px;" ><b>NOTE</b></p>  
<br>    

<p style="font-size:18px;" >
If you'd like to <a href="https://github.com/atefbadji/cfPWV_Microstructure-Perfusion_CorpusCallosum-/archive/master.zip">download GitHub repository</a> and run it locally, please use the <code>cfPWV_CC_notebook_local.ipynb</code> and make sure that you have installed all the dependencies.</p>
</div>

# <i class="fa fa-compass" style="font-size: 1.5em;"></i><left> &nbsp;Read data.</left> 

In [ ]:
data <- read.csv('20190821_Data.csv')
colnames(data) <- c('ID','sex','UnderTreatment','age','COND96',
                    'cfPWV','SBP','DBP','PP','AIx','CBF_CC','ICVF_CC','ICVF_CI',
                    'ICVF_CR','ICVF_SLF','ISOVF_CC','ISOVF_CI','ISOVF_CR','ISOVF_SLF',
                    'ISOVF_CC_genu','ISOVF_CC_body','ISOVF_CC_splenium','ICVF_CC_genu','ICVF_CC_body',
                    'ICVF_CC_splenium','CBF_CC_genu','CBF_CC_body','CBF_CC_splenium','education',
                    'TMTBA')
data$COND96<-as.factor(data$COND96)
#If you want to visualise the dataframe, uncomment the line below
#data or print(data)
#if you want to see the dimention of the data (numer of lines, number of colones), uncomment the line below
dim(data) 


# <i class="fa fa-compass" style="font-size: 1.5em;"></i><left> &nbsp; Patients characteristics. </left> 

<i class="fa fa-info-circle" style="font-size: 1.5em;"></i> &nbsp; The following table shows participants MRI characteristics including a comparison among drug naive participants and those on antihypertensive treatments. A more complete table including a comparison of all demographics, cardiovascular and neuropsychological measures between drug-naive participants and participants on antihypertensive medication can be found in a previous publication (https://www.ncbi.nlm.nih.gov/pubmed/30448213).

In [ ]:
mean(data$ISOVF_CC)
sd(data$ISOVF_CC)
ISOVF_CC_treated <-data$ISOVF_CC[data$UnderTreatment==1]
ISOVF_CC_untreated <-data$ISOVF_CC[data$UnderTreatment==0]
mean(ISOVF_CC_treated)
sd(ISOVF_CC_treated)
mean(ISOVF_CC_untreated)
sd(ISOVF_CC_untreated)
res1 <- wilcox.test(ISOVF_CC_treated, ISOVF_CC_untreated)
res1
mean(data$ICVF_CC)
sd(data$ICVF_CC)
ICVF_CC_treated <-data$ICVF_CC[data$UnderTreatment==1]
ICVF_CC_untreated <-data$ICVF_CC[data$UnderTreatment==0]
mean(ICVF_CC_treated)
sd(ICVF_CC_treated)
mean(ICVF_CC_untreated)
sd(ICVF_CC_untreated)
res2 <- wilcox.test(ICVF_CC_treated, ICVF_CC_untreated)
res2
mean(data$CBF_CC)
sd(data$CBF_CC)
CBF_CC_treated <-data$CBF_CC[data$UnderTreatment==1]
CBF_CC_untreated <-data$CBF_CC[data$UnderTreatment==0]
mean(CBF_CC_treated)
sd(CBF_CC_treated)
mean(CBF_CC_untreated)
sd(CBF_CC_untreated)
res3 <- wilcox.test(CBF_CC_treated, CBF_CC_untreated)
res3


# <i class="fa fa-compass" style="font-size: 1.5em;"></i><left> &nbsp; Remove outliers. </left> 


In [ ]:
# copy the data to dsBase.iqr
dsBase.iqr <- data 
# create a variable/collection of the column names you want to remove outliers on.
vars <- c("cfPWV", "TMTBA", "ISOVF_CC", "ICVF_CC", "CBF_CC")
# create a variable to store the row id's to be removed
Outliers <- c()
# loop through the list of columns you specified
for (i in vars) {
# Get the Min/Max values
max <- quantile(dsBase.iqr[,i],0.75, na.rm=TRUE) + (IQR(dsBase.iqr[,i], na.rm=TRUE) * 1.5 )
min <- quantile(dsBase.iqr[,i],0.25, na.rm=TRUE) - (IQR(dsBase.iqr[,i], na.rm=TRUE) * 1.5 )
# Get the id's using which
idx <- which(dsBase.iqr[,i] < min | dsBase.iqr[,i] > max)
# Output the number of outliers in each variable
print(paste(i, length(idx), sep=''))
# Append the outliers list
Outliers <- c(Outliers, idx) 
}
cat('Outliers are: ', Outliers)

In [ ]:
# Remove the outliers
dsBase.iqr <- dsBase.iqr[-Outliers,]
# visualise the new dataframe without outliers
#dsBase.iqr
print.summary = TRUE


# <i class="fa fa-compass" style="font-size: 1.5em;"></i><left> &nbsp; Statistical analyses. </left> 

### <font color="Teal"> STEP 1: Partial correlation between arterial stiffness (cfPWV), the microstructure (ISOVF_CC, ICVF_CC) and the perfusion (CBF_CC) of the corpus callosum</font>





In [ ]:
#1.1: Partial correlation between cfPWV and ISOVF_CC
y.data <- data.frame(dsBase.iqr$age,dsBase.iqr$sex)
pcor.test(dsBase.iqr$cfPWV, dsBase.iqr$ISOVF_CC, y.data, method = c("spearman"))
library(plotly)
gplot1 <- ggplot(dsBase.iqr, aes(x=dsBase.iqr$cfPWV, y=dsBase.iqr$ISOVF_CC)) + geom_point(aes(shape=COND96, color=COND96, size=10)) + geom_smooth(method=lm, se=FALSE) + theme_bw()+ theme(text = element_text(size=20)) + xlab("cfPWV") + ylab("ISOVF") 
# To visualise gplot1 with an interactive plot, uncomment the line below
#ggplotly(gplot1)

#1.2: Partial correlation between cfPWV and ICVF_CC
y.data <- data.frame(dsBase.iqr$age,dsBase.iqr$sex)
pcor.test(dsBase.iqr$cfPWV, dsBase.iqr$ICVF_CC, y.data, method = c("spearman"))
library(plotly)
gplot2 <- ggplot(dsBase.iqr, aes(x=dsBase.iqr$cfPWV, y=dsBase.iqr$ICVF_CC)) + geom_point(aes(shape=COND96, color=COND96, size=10)) + geom_smooth(method=lm, se=FALSE) +theme_bw() + theme(text = element_text(size=20)) + xlab("cfPWV") + ylab("ICVF")
# To visualise gplot2 with an interactive plot, uncomment the line below
#ggplotly(gplot2)

#1.3: Partial correlation between cfPWV and CBF_CC
y.data <- data.frame(dsBase.iqr$age,dsBase.iqr$sex)
pcor.test(dsBase.iqr$cfPWV, dsBase.iqr$CBF_CC, y.data, method = c("spearman"))
library(plotly)
gplot3 <- ggplot(dsBase.iqr, aes(x=dsBase.iqr$cfPWV, y=dsBase.iqr$CBF_CC)) + geom_point(aes(shape=COND96, color=COND96, size=10)) + geom_smooth(method=lm, se=FALSE) + theme_bw() + theme(text = element_text(size=20))+ xlab("cfPWV") + ylab("CBF")
# To visualise gplot3 with an interactive plot, uncomment the line below
ggplotly(gplot3)

### <font color="Red"> Figure 1 </font>

In [ ]:
options(repr.plot.width=23, repr.plot.height=4)
grid.arrange(gplot1, gplot2, gplot3, nrow=1, ncol=3) 

### <font color="Teal"> STEP 2: Partial correlation between arterial stiffness (cfPWV), the microstructure and perfusion of the genu, body and splenium of the corpus callosum</font>




In [ ]:
#2.1

#Partial correlation between cfPWV and ISOVF_CC_genu
y.data <- data.frame(dsBase.iqr$age,dsBase.iqr$sex)
pcor.test(dsBase.iqr$cfPWV, dsBase.iqr$ISOVF_CC_genu, y.data, method = c("spearman"))
library(plotly)
gplot1 <- ggplot(dsBase.iqr, aes(x=dsBase.iqr$cfPWV, y=dsBase.iqr$ISOVF_CC_genu)) + geom_point(aes(shape=COND96, color=COND96, size=10)) + geom_smooth(method=lm, se=FALSE) +theme_bw() + theme(text = element_text(size=20)) + xlab("cfPWV") + ylab("ISOVF_genu")
#To visualize gplot1 with an interactive plot, uncomment the line below
#ggplotly(gplot1)

#Partial correlation between cfPWV and ISOVF_CC_body
y.data <- data.frame(dsBase.iqr$age,dsBase.iqr$sex)
pcor.test(dsBase.iqr$cfPWV, dsBase.iqr$ISOVF_CC_body, y.data, method = c("spearman"))
library(plotly)
gplot2 <- ggplot(dsBase.iqr, aes(x=dsBase.iqr$cfPWV, y=dsBase.iqr$ISOVF_CC_body)) + geom_point(aes(shape=COND96, color=COND96, size=10)) + geom_smooth(method=lm, se=FALSE) +theme_bw() + theme(text = element_text(size=20)) + xlab("cfPWV") + ylab("ISOVF+body")
#To visualize gplot2 with an interactive plot, uncomment the line below
#ggplotly(gplot2)

#Partial correlation between cfPWV and ISOVF_CC_splenium
y.data <- data.frame(dsBase.iqr$age,dsBase.iqr$sex)
pcor.test(dsBase.iqr$cfPWV, dsBase.iqr$ISOVF_CC_splenium, y.data, method = c("spearman"))
library(plotly)
gplot3 <- ggplot(dsBase.iqr, aes(x=dsBase.iqr$cfPWV, y=dsBase.iqr$ISOVF_CC_splenium)) + geom_point(aes(shape=COND96, color=COND96, size=10)) + geom_smooth(method=lm, se=FALSE) +theme_bw() + theme(text = element_text(size=20))+ xlab("cfPWV") + ylab("ISOVF_splenium")
#To visualize gplot3 with an interactive plot, uncomment the line below
#ggplotly(gplot3)

In [ ]:
#2.2

#Partial correlation between cfPWV and ICVF_CC_genu
y.data <- data.frame(dsBase.iqr$age,dsBase.iqr$sex)
pcor.test(dsBase.iqr$cfPWV, dsBase.iqr$ICVF_CC_genu, y.data, method = c("spearman"))
library(plotly)
gplot4 <- ggplot(dsBase.iqr, aes(x=dsBase.iqr$cfPWV, y=dsBase.iqr$ICVF_CC_genu)) + geom_point(aes(shape=COND96, color=COND96, size=10)) + geom_smooth(method=lm, se=FALSE) +theme_bw() + theme(text = element_text(size=20))+ xlab("cfPWV") + ylab("ICVF_genu")
#To visualize gplot4 with an interactive plot, uncomment the line below
#ggplotly(gplot4)

#Partial correlation between cfPWV and ICVF_CC_body
y.data <- data.frame(dsBase.iqr$age,dsBase.iqr$sex)
pcor.test(dsBase.iqr$cfPWV, dsBase.iqr$ICVF_CC_body, y.data, method = c("spearman"))
library(plotly)
gplot5 <- ggplot(dsBase.iqr, aes(x=dsBase.iqr$cfPWV, y=dsBase.iqr$ICVF_CC_body)) + geom_point(aes(shape=COND96, color=COND96, size=10)) + geom_smooth(method=lm, se=FALSE) +theme_bw() + theme(text = element_text(size=20)) + xlab("cfPWV") + ylab("ICVF_body") 
#To visualize gplot5 with an interactive plot, uncomment the line below
#ggplotly(gplot5)

#Partial correlation between cfPWV and ICVF_CC_splenium
y.data <- data.frame(dsBase.iqr$age,dsBase.iqr$sex)
pcor.test(dsBase.iqr$cfPWV, dsBase.iqr$ICVF_CC_splenium, y.data, method = c("spearman"))
library(plotly)
gplot6 <- ggplot(dsBase.iqr, aes(x=dsBase.iqr$cfPWV, y=dsBase.iqr$ICVF_CC_splenium)) + geom_point(aes(shape=COND96, color=COND96, size=10)) + geom_smooth(method=lm, se=FALSE) +theme_bw() + theme(text = element_text(size=20)) + xlab("cfPWV") + ylab("ICVF_splenium")
#To visualize gplot6 with an interactive plot, uncomment the line below
#ggplotly(gplot6)

In [ ]:
#2.3

#Partial correlation between cfPWV and CBF_CC_genu
y.data <- data.frame(dsBase.iqr$age,dsBase.iqr$sex)
pcor.test(dsBase.iqr$cfPWV, dsBase.iqr$CBF_CC_genu, y.data, method = c("spearman"))
library(plotly)
gplot7 <- ggplot(dsBase.iqr, aes(x=dsBase.iqr$cfPWV, y=dsBase.iqr$CBF_CC_genu)) + geom_point(aes(shape=COND96, color=COND96, size=10)) + geom_smooth(method=lm, se=FALSE) +theme_bw() + theme(text = element_text(size=20)) + xlab("cfPWV") + ylab("CBF_genu")
#To visualize gplot7 with an interactive plot, uncomment the line below
#ggplotly(gplot7)

#Partial correlation between cfPWV and CBF_CC_body
y.data <- data.frame(dsBase.iqr$age,dsBase.iqr$sex)
pcor.test(dsBase.iqr$cfPWV, dsBase.iqr$CBF_CC_body, y.data, method = c("spearman"))
library(plotly)
gplot8 <- ggplot(dsBase.iqr, aes(x=dsBase.iqr$cfPWV, y=dsBase.iqr$CBF_CC_body)) + geom_point(aes(shape=COND96, color=COND96, size=10)) + geom_smooth(method=lm, se=FALSE) +theme_bw() + theme(text = element_text(size=20))+ xlab("cfPWV") + ylab("CBF_body")
#To visualize gplot8 with an interactive plot, uncomment the line below
#ggplotly(gplot8)

#Partial correlation between cfPWV and CBF_CC_splenium
y.data <- data.frame(dsBase.iqr$age,dsBase.iqr$sex)
pcor.test(dsBase.iqr$cfPWV, dsBase.iqr$CBF_CC_splenium, y.data, method = c("spearman"))
library(plotly)
gplot9 <- ggplot(dsBase.iqr, aes(x=dsBase.iqr$cfPWV, y=dsBase.iqr$CBF_CC_splenium)) + geom_point(aes(shape=COND96, color=COND96, size=10)) + geom_smooth(method=lm, se=FALSE) +theme_bw() + theme(text = element_text(size=20)) + xlab("cfPWV") + ylab("CBF_splenium")


### <font color="Red"> Figure 2 </font>

In [ ]:
#To visualize gplot9 with an interactive plot, uncomment the line below
#ggplotly(gplot9)
options(repr.plot.width=23, repr.plot.height=10)
grid.arrange(gplot1, gplot2, gplot3, gplot4, gplot5, gplot6, gplot7, gplot8, gplot9, nrow=3, ncol=3)

### <font color="Teal"> STEP 3: Stepwise linear regression among indexes of vascular aging (cfPWV, 24h SBP, 24h PP and AIx) on our MRI metrics (ISOVF_CC, ICVF_CC, CBF_CC)</font>


In [ ]:
#3.1: Stepwise linear regression with ISOVF_CC as dependent variable
model_ISOVF <- lm(dsBase.iqr$ISOVF_CC ~ dsBase.iqr$age + dsBase.iqr$sex + dsBase.iqr$cfPWV + dsBase.iqr$SBP + dsBase.iqr$PP + dsBase.iqr$AIx,data= dsBase.iqr)
ols_step_forward_p(model_ISOVF, details=TRUE)
summary(model_ISOVF)

In [ ]:
# 3.2: Stepwise linear regression with ICVF_CC as dependent variable
model_ICVF = lm(dsBase.iqr$ICVF_CC ~ dsBase.iqr$age + dsBase.iqr$sex + dsBase.iqr$cfPWV + dsBase.iqr$SBP + dsBase.iqr$PP + dsBase.iqr$AIx,data= dsBase.iqr)
ols_step_forward_aic(model_ICVF, details=TRUE)
summary(stepwise)

In [ ]:
#3.3: Stepwise linear regression with CBF_CC as dependent variable
model_CBF = lm(dsBase.iqr$CBF_CC ~ dsBase.iqr$age + dsBase.iqr$sex + dsBase.iqr$cfPWV + dsBase.iqr$SBP + dsBase.iqr$PP + dsBase.iqr$AIx,data= dsBase.iqr)
ols_step_forward_aic(model_CBF, details=TRUE)
summary(stepwise)

### <font color="Teal"> STEP 4: Partial correlation between our MRI metrics (ISOVF_CC, ICVF_CC and CBF_CC) and the performance in cognitive flexibility (TMTB-A)</font>


In [ ]:
#4.1: Partial correlation between ISOVF_CC and TMTB-A
y.data <- data.frame(dsBase.iqr$age,dsBase.iqr$sex,dsBase.iqr$education)
pcor.test(dsBase.iqr$ISOVF_CC, dsBase.iqr$TMTBA, y.data, method = c("spearman"))
library(plotly)
gplot1 <- ggplot(dsBase.iqr, aes(x=dsBase.iqr$ISOVF_CC, y=dsBase.iqr$TMTBA)) + geom_point(aes(shape=COND96, color=COND96, size=10)) + geom_smooth(method=lm, se=FALSE) + theme_bw() + theme(text = element_text(size=20)) + xlab("ISOVF") + ylab("TMTB-A") 
#To visualize gplot1 with an interactive plot, uncomment the line below
#ggplotly(gplot1)

#4.2: Partial correlation between ICVF_CC and TMTB-A
y.data <- data.frame(dsBase.iqr$age,dsBase.iqr$sex,dsBase.iqr$education)
pcor.test(dsBase.iqr$ICVF_CC, dsBase.iqr$TMTBA, y.data, method = c("spearman"))
library(plotly)
gplot2 <- ggplot(dsBase.iqr, aes(x=dsBase.iqr$ICVF_CC, y=dsBase.iqr$TMTBA)) + geom_point(aes(shape=COND96, color=COND96, size=10)) + geom_smooth(method=lm, se=FALSE) +theme_bw() + theme(text = element_text(size=20)) + xlab("ICVF") + ylab("TMTB-A") 
#To visualize gplot2 with an interactive plot, uncomment the line below
#ggplotly(gplot2)

#4.3: Partial correlation between CBF_CC and TMTB-A
y.data <- data.frame(dsBase.iqr$age,dsBase.iqr$sex,dsBase.iqr$education)
pcor.test(dsBase.iqr$CBF_CC, dsBase.iqr$TMTBA, y.data, method = c("spearman"))
library(plotly)
gplot3 <- ggplot(dsBase.iqr, aes(x=dsBase.iqr$CBF_CC, y=dsBase.iqr$TMTBA)) + geom_point(aes(shape=COND96, color=COND96, size=10)) + geom_smooth(method=lm, se=FALSE) +theme_bw() + theme(text = element_text(size=20)) + xlab("CBF") + ylab("TMTB-A") 
#To visualize gplot3 with an interactive plot, uncomment the line below
#ggplotly(gplot3)

### <font color="Red"> Figure 3 </font>

In [ ]:
options(repr.plot.width=23, repr.plot.height=4)
grid.arrange(gplot1, gplot2, gplot3, nrow=1, ncol=3)

### <font color="Teal"> STEP 5: Partial correlation between the microstructure and perfusion of the genu, body and splenium of the corpus callosum with the performance in cognitive flexibility</font>


In [ ]:
#5.1

#Partial correlation between ISOVF_CC_genu and TMTB-A
y.data <- data.frame(dsBase.iqr$age,dsBase.iqr$sex)
pcor.test(dsBase.iqr$ISOVF_CC_genu, dsBase.iqr$TMTBA, y.data, method = c("spearman"))
library(plotly)
gplot1 <- ggplot(dsBase.iqr, aes(x=dsBase.iqr$ISOVF_CC_genu, y=dsBase.iqr$TMTBA)) + geom_point(aes(shape=COND96, color=COND96, size=10)) + geom_smooth(method=lm, se=FALSE) + theme_bw() + theme(text = element_text(size=20)) + xlab("ISOVF_genu") + ylab("TMTB-A")   
#To visualize gplot1 with an interactive plot, uncomment the line below
#ggplotly(gplot1)

#Partial correlation between ISOVF_CC_body and TMTB-A
y.data <- data.frame(dsBase.iqr$age,dsBase.iqr$sex)
pcor.test(dsBase.iqr$ISOVF_CC_body, dsBase.iqr$TMTBA, y.data, method = c("spearman"))
library(plotly)
gplot2 <- ggplot(dsBase.iqr, aes(x=dsBase.iqr$ISOVF_CC_body, y=dsBase.iqr$TMTBA)) + geom_point(aes(shape=COND96, color=COND96, size=10)) + geom_smooth(method=lm, se=FALSE) + theme_bw() + theme(text = element_text(size=20))+ xlab("ISOVF_body") + ylab("TMTB-A") 
#To visualize gplot2 with an interactive plot, uncomment the line below
#ggplotly(gplot2)

#Partial correlation between ISOVF_CC_splenium and TMTB-A
y.data <- data.frame(dsBase.iqr$age,dsBase.iqr$sex)
pcor.test(dsBase.iqr$ISOVF_CC_splenium, dsBase.iqr$TMTBA, y.data, method = c("spearman"))
library(plotly)
gplot3 <- ggplot(dsBase.iqr, aes(x=dsBase.iqr$ISOVF_CC_splenium, y=dsBase.iqr$TMTBA)) + geom_point(aes(shape=COND96, color=COND96, size=10)) + geom_smooth(method=lm, se=FALSE) + theme_bw() + theme(text = element_text(size=20)) + xlab("ISOVF_splenium") + ylab("TMTB-A") 
#To visualize gplot3 with an interactive plot, uncomment the line below
#ggplotly(gplot3)

In [ ]:
#5.2

#Partial correlation between ICVF_CC_genu and TMTB-A
y.data <- data.frame(dsBase.iqr$age,dsBase.iqr$sex)
pcor.test(dsBase.iqr$ICVF_CC_genu, dsBase.iqr$TMTBA, y.data, method = c("spearman"))
library(plotly)
gplot4 <- ggplot(dsBase.iqr, aes(x=dsBase.iqr$ICVF_CC_genu, y=dsBase.iqr$TMTBA)) + geom_point(aes(shape=COND96, color=COND96, size=10)) + geom_smooth(method=lm, se=FALSE) + theme_bw() + theme(text = element_text(size=20)) + xlab("ICVF_genu") + ylab("TMTB-A")   
#To visualize gplot4 with an interactive plot, uncomment the line below
#ggplotly(gplot4)

#Partial correlation between ICVF_CC_body and TMTB-A
y.data <- data.frame(dsBase.iqr$age,dsBase.iqr$sex)
pcor.test(dsBase.iqr$ICVF_CC_body, dsBase.iqr$TMTBA, y.data, method = c("spearman"))
library(plotly)
gplot5 <- ggplot(dsBase.iqr, aes(x=dsBase.iqr$ICVF_CC_body, y=dsBase.iqr$TMTBA)) + geom_point(aes(shape=COND96, color=COND96, size=10)) + geom_smooth(method=lm, se=FALSE) + theme_bw() + theme(text = element_text(size=20))+ xlab("ICVF_body") + ylab("TMTB-A") 
#To visualize gplot5 with an interactive plot, uncomment the line below
#ggplotly(gplot5)

#Partial correlation between ICVF_CC_splenium and TMTB-A
y.data <- data.frame(dsBase.iqr$age,dsBase.iqr$sex)
pcor.test(dsBase.iqr$ICVF_CC_splenium, dsBase.iqr$TMTBA, y.data, method = c("spearman"))
library(plotly)
gplot6 <- ggplot(dsBase.iqr, aes(x=dsBase.iqr$ICVF_CC_splenium, y=dsBase.iqr$TMTBA)) + geom_point(aes(shape=COND96, color=COND96, size=10)) + geom_smooth(method=lm, se=FALSE) + theme_bw() + theme(text = element_text(size=20)) + xlab("ICVF_splenium") + ylab("TMTB-A") 
#To visualize gplot6 with an interactive plot, uncomment the line below
#ggplotly(gplot6)

In [ ]:
#5.3

#Partial correlation between CBF_CC_genu and TMTB-A
y.data <- data.frame(dsBase.iqr$age,dsBase.iqr$sex)
pcor.test(dsBase.iqr$CBF_CC_genu, dsBase.iqr$TMTBA, y.data, method = c("spearman"))
library(plotly)
gplot7 <- ggplot(dsBase.iqr, aes(x=dsBase.iqr$CBF_CC_genu, y=dsBase.iqr$TMTBA)) + geom_point(aes(shape=COND96, color=COND96, size=10)) + geom_smooth(method=lm, se=FALSE) + theme_bw()+ theme(text = element_text(size=20)) + xlab("CBF_genu") + ylab("TMTB-A")   
#To visualize gplot7 with an interactive plot, uncomment the line below
#ggplotly(gplot7)

#Partial correlation between CBF_CC_body and TMTB-A
y.data <- data.frame(dsBase.iqr$age,dsBase.iqr$sex)
pcor.test(dsBase.iqr$CBF_CC_body, dsBase.iqr$TMTBA, y.data, method = c("spearman"))
library(plotly)
gplot8 <- ggplot(dsBase.iqr, aes(x=dsBase.iqr$CBF_CC_body, y=dsBase.iqr$TMTBA)) + geom_point(aes(shape=COND96, color=COND96, size=10)) + geom_smooth(method=lm, se=FALSE) + theme_bw() + theme(text = element_text(size=20))+ xlab("CBF_body") + ylab("TMTB-A") 
#To visualize gplot8 with an interactive plot, uncomment the line below
#ggplotly(gplot8)

#Partial correlation between ICVF_CC_splenium and TMTB-A
y.data <- data.frame(dsBase.iqr$age,dsBase.iqr$sex)
pcor.test(dsBase.iqr$CBF_CC_splenium, dsBase.iqr$TMTBA, y.data, method = c("spearman"))
library(plotly)
gplot9 <- ggplot(dsBase.iqr, aes(x=dsBase.iqr$CBF_CC_splenium, y=dsBase.iqr$TMTBA)) + geom_point(aes(shape=COND96, color=COND96, size=10)) + geom_smooth(method=lm, se=FALSE) + theme_bw() + theme(text = element_text(size=20)) + xlab("CBF_splenium") + ylab("TMTB-A") 
#To visualize gplot9 with an interactive plot, uncomment the line below
#ggplotly(gplot9)


### <font color="Red"> Figure 4 </font>

In [ ]:
options(repr.plot.width=23, repr.plot.height=10)
grid.arrange(gplot1, gplot2, gplot3, gplot4, gplot5, gplot6, gplot7, gplot8, gplot9, nrow=3, ncol=3)

<img src="copyright_neuropoly.jpg">"